In [1]:
!pip3 install torch
!pip3 install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 4.4 MB/s eta 0:00:0000:0100:01m
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 3.8 MB/s eta 0:00:0000:01
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.2 MB/s eta 0:00:0000:0100:010m
  Using cached kiwisolver-1.4.4-cp311-cp311-macosx_11_0_arm64.whl (63 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 4.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [344]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [345]:
names_list = open('names.txt','r').read().split('\n')

In [346]:
print(names_list[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [347]:
char_to_int = {}
int_to_char = {}

char_to_int['.'] = 0
int_to_char[0] = '.'

for char_num in range(ord('a'),ord('z')+1):
    integer_representation = char_num-ord('a')+1
    char_to_int[chr(char_num)] = integer_representation
    int_to_char[integer_representation] = chr(char_num)


In [348]:
#hyperparameters

embed_size = 6

state_size = 20

pre_hidden_size = 200

hidden_size = 100

vocab_size = 28

In [349]:
import math
char_vector_size = 2
input_contexts , input_labels = [[] for _ in range(5)], [[] for _ in range(5)]

for name in names_list:
    log_padded_length = math.ceil(math.log(len(name))/math.log(2))
    padded_length = int(2**log_padded_length)
    context = [char_to_int[char] for char in name]  + [0] + [vocab_size-1]*(padded_length-len(name)) 
    input_contexts[log_padded_length].append(context)
#   instead of using labels, we can compare the result to the character vectors to find a likely match

In [350]:
freqs = [0 for i in range(vocab_size-1)]

for log_len in range(1,5):
    contexts = input_contexts[log_len]
    for c in contexts:
        for int_char in c:
            if int_char<vocab_size-1:
                freqs[int_char] += 1
                
freqs = torch.tensor(freqs+[0],dtype=torch.float64)
freqs/=freqs.sum()

freqs

tensor([0.1404, 0.1485, 0.0116, 0.0155, 0.0241, 0.0895, 0.0040, 0.0084, 0.0334,
        0.0776, 0.0127, 0.0221, 0.0612, 0.0291, 0.0803, 0.0348, 0.0045, 0.0012,
        0.0557, 0.0355, 0.0244, 0.0137, 0.0113, 0.0041, 0.0031, 0.0428, 0.0105,
        0.0000], dtype=torch.float64)

In [351]:
word = input_contexts[2][1]

In [370]:
# parameters
all_char_embeds = torch.randn((vocab_size,embed_size))

all_char_embeds[vocab_size-1] = 10**10

w_ep = torch.randn((embed_size, pre_hidden_size)) * 0.5

w_sp = torch.randn((state_size, pre_hidden_size)) * 0.5
w_so = torch.randn((state_size, embed_size)) *0.5

w_ph = torch.randn((pre_hidden_size, hidden_size)) * 0.5

w_hs = torch.randn((hidden_size, state_size)) *0.5


b_p = torch.zeros((pre_hidden_size,))
b_h = torch.zeros((hidden_size,))
b_s = torch.zeros((state_size,))
b_o = torch.zeros((embed_size,))

batch_norm_layer = nn.BatchNorm1d(hidden_size)

init_state = torch.randn((state_size,))

network_params = [init_state,w_ep,w_sp,w_ph,w_so,w_hs,b_p,b_h,b_s,b_o,all_char_embeds]

network_params += [p for p in batch_norm_layer.parameters()]

for p in network_params:
    p.requires_grad = True

In [372]:
def process_state_and_embed(states , embeds, hiddens, outputs, idx):
    pre_hidden = 2*torch.sigmoid((states[idx] @ w_sp) + (embeds[idx] @ w_ep) + b_p) -1    
    hiddens[idx] = batch_norm_layer(pre_hidden @ w_ph + b_h)
    
    states[idx+1] = 2*torch.sigmoid(hiddens[idx] @ w_hs + b_s) - 1
    outputs[idx] = states[idx] @ w_so + b_o

In [356]:
batch_size = 64

for i in range(len(input_contexts)):
    input_contexts[i] = torch.tensor(input_contexts[i])

/var/folders/qj/ks60gt8d7bbbzmtsh2l4f8y00000gn/T/ipykernel_29664/955914535.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_contexts[i] = torch.tensor(input_contexts[i])


In [392]:
loss_tot = 0

loops = 500000
check_every = 500

# training loop
for i in range(loops):
    log_idx = random.randint(1,4)
#     log_idx = 4
    batch_indices = torch.randint(0,input_contexts[log_idx].shape[0],(batch_size,))

    batch = input_contexts[log_idx][batch_indices]

    word_length = len(batch[0])

    embeds = torch.transpose(all_char_embeds[batch],0,1)
    
    factors = torch.where(freqs > 0,1/(freqs+1e-18),0)
    
    factors[:-1] /= factors[:-1].mean()
    
    adjustment_factors = torch.transpose(factors[batch],0,1)
    
    adjustment_factors = adjustment_factors.view((word_length,batch_size,1))
    
    adjustment_factors[:-1] /= adjustment_factors[:-1].mean()
#     print(adjustment_factors , embeds.shape)

    states = [init_state] + [torch.zeros((batch_size,state_size,)) for _ in range(word_length-1)]

    hiddens = [torch.zeros((batch_size,hidden_size,)) for _ in range(word_length-1)]

    outputs = torch.zeros((word_length,batch_size,embed_size))

    for j in range(word_length-1):
        process_state_and_embed(states,embeds ,hiddens,outputs,j)

    outputs[-1] = states[-1] @ w_so + b_o
    
    distances_sqd = (torch.cdist(outputs,all_char_embeds[:-1]) ** 2).sum(2,keepdim=True)
        
    filtered = torch.where(embeds != 1e10, ((outputs-embeds)**2 / distances_sqd), 0)
    
    ones = torch.where(embeds != 1e10, 1, 0)
    
    loss = (filtered*adjustment_factors).sum()
    
    loss.backward()
    
    for p in network_params:
        p.data -= 1e-3*p.grad
        p.grad = None
        
    loss_tot+= loss/ adjustment_factors.sum()
    if i % check_every==0 and i>0:
        print(f"{i} of {loops}: " , loss_tot / (check_every))
        loss_tot = 0
#     print('o',outputs)
#     print('e',embeds)
#     print('wl',word_length)

500 of 500000:  tensor(0.0151, dtype=torch.float64, grad_fn=<DivBackward0>)
1000 of 500000:  tensor(0.0152, dtype=torch.float64, grad_fn=<DivBackward0>)
1500 of 500000:  tensor(0.0152, dtype=torch.float64, grad_fn=<DivBackward0>)
2000 of 500000:  tensor(0.0151, dtype=torch.float64, grad_fn=<DivBackward0>)
2500 of 500000:  tensor(0.0150, dtype=torch.float64, grad_fn=<DivBackward0>)
3000 of 500000:  tensor(0.0152, dtype=torch.float64, grad_fn=<DivBackward0>)
3500 of 500000:  tensor(0.0153, dtype=torch.float64, grad_fn=<DivBackward0>)
4000 of 500000:  tensor(0.0151, dtype=torch.float64, grad_fn=<DivBackward0>)
4500 of 500000:  tensor(0.0150, dtype=torch.float64, grad_fn=<DivBackward0>)
5000 of 500000:  tensor(0.0149, dtype=torch.float64, grad_fn=<DivBackward0>)
5500 of 500000:  tensor(0.0149, dtype=torch.float64, grad_fn=<DivBackward0>)
6000 of 500000:  tensor(0.0152, dtype=torch.float64, grad_fn=<DivBackward0>)
6500 of 500000:  tensor(0.0153, dtype=torch.float64, grad_fn=<DivBackward0>)


53500 of 500000:  tensor(0.0140, dtype=torch.float64, grad_fn=<DivBackward0>)
54000 of 500000:  tensor(0.0142, dtype=torch.float64, grad_fn=<DivBackward0>)
54500 of 500000:  tensor(0.0142, dtype=torch.float64, grad_fn=<DivBackward0>)
55000 of 500000:  tensor(0.0143, dtype=torch.float64, grad_fn=<DivBackward0>)
55500 of 500000:  tensor(0.0138, dtype=torch.float64, grad_fn=<DivBackward0>)
56000 of 500000:  tensor(0.0141, dtype=torch.float64, grad_fn=<DivBackward0>)
56500 of 500000:  tensor(0.0139, dtype=torch.float64, grad_fn=<DivBackward0>)
57000 of 500000:  tensor(0.0136, dtype=torch.float64, grad_fn=<DivBackward0>)
57500 of 500000:  tensor(0.0138, dtype=torch.float64, grad_fn=<DivBackward0>)
58000 of 500000:  tensor(0.0139, dtype=torch.float64, grad_fn=<DivBackward0>)
58500 of 500000:  tensor(0.0140, dtype=torch.float64, grad_fn=<DivBackward0>)
59000 of 500000:  tensor(0.0139, dtype=torch.float64, grad_fn=<DivBackward0>)
59500 of 500000:  tensor(0.0137, dtype=torch.float64, grad_fn=<D

KeyboardInterrupt: 

In [405]:
def predict(word):
    word_length = len(word)
    
    embeds = all_char_embeds[word].view(word_length,1,embed_size)
    
    states = [init_state]+[torch.zeros((1,state_size,)) for _ in range(word_length-1)]

    hiddens = [torch.zeros((1,hidden_size,)) for _ in range(word_length-1)]

    outputs = torch.zeros((word_length,1,embed_size))

    for i in range(word_length-1):
        process_state_and_embed(states,embeds ,hiddens,outputs,i)

    return states[-1] @ w_so + b_o


def get_next(text):
    word = [char_to_int[c] for c in text] +[0]

    pred = predict(word)

    pred = pred.view(1,embed_size)
    dists = torch.cdist(pred,all_char_embeds[:-1])
    dists = dists.view(-1)
    dists = torch.exp(-10*dists)
    dists/=dists.sum()
    sampled = (torch.multinomial(dists,1)).item()
    return int_to_char[sampled]

In [ ]:
start_st = ''

batch_norm_layer.eval()

for _ in range(100):
    st = start_st
    curr = 'a'
    while curr!='.':
        curr = get_next(st)
        st+=curr
    print(st)

batch_norm_layer.train(True)

In [403]:
((all_char_embeds[char_to_int['k']]-all_char_embeds[char_to_int['c']])**2).sum()

tensor(1.0844, grad_fn=<SumBackward0>)

In [395]:
all_char_embeds

tensor([[ 5.9781e+00, -2.0762e+00, -3.4334e+00,  2.4585e+00,  1.4191e+00,
          6.3284e-01],
        [ 2.8782e+00, -1.2092e+00, -1.2280e+00,  1.9302e+00,  1.4470e+00,
          1.0097e+00],
        [-1.4000e+00,  9.5057e-01,  5.8750e-01, -4.9056e-01, -4.7422e-01,
          1.9161e-01],
        [-1.2603e+00,  7.0542e-01,  7.4141e-01, -5.2271e-01, -7.6276e-01,
          9.6872e-02],
        [-1.3179e+00,  6.0693e-01,  4.3269e-01, -2.9754e-01, -5.7641e-01,
          7.0558e-02],
        [ 2.5471e+00, -8.5447e-01, -1.3418e+00,  1.7515e+00,  1.3425e+00,
          9.6918e-01],
        [-1.7320e+00,  9.7192e-01,  8.1608e-01, -9.3318e-01, -8.1058e-01,
          7.7136e-02],
        [-1.5648e+00,  7.5046e-01,  7.1344e-01, -5.1856e-01, -7.9058e-01,
          3.2312e-01],
        [ 1.2899e+00, -3.9766e-01, -3.1215e-01,  1.1563e+00,  5.1118e-01,
          3.5854e-01],
        [ 2.7570e+00, -2.2321e+00, -1.0240e+00,  1.7206e+00,  1.5739e+00,
          7.8637e-01],
        [-1.5944e+00,  7.5318e